In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import warnings
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import string
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    return soup


def extract_links(url):
    soup = get_soup(url)
    aitems = []
    for h3 in soup.find_all("h3"):
        try:
            a = h3.a["href"]
        except:
            KeyError
        aitems.append(a)
    return aitems


all_links1 = [
    f"https://xwatch.vn/dong-ho-nam-pc85-page{idx_page}.html"
    for idx_page in range(1, 206)
]
all_links2 = [
    f"https://xwatch.vn/dong-ho-nu-pc86-page{idx_page}.html"
    for idx_page in range(1, 107)
]

all_links = []

with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
    futures = [executor.submit(extract_links, url) for url in all_links1 + all_links2]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

all_links = [link for sublist in results for link in sublist]

In [3]:
all_links = list(set(all_links))

all_links

['https://xwatch.vn/dong-ho-nam/bm7141-51a-p930596.html',
 'https://xwatch.vn/dong-ho-nam/dong-ho-seiko-srpd71k1-p957197.html',
 'https://xwatch.vn/dong-ho-nam/ssb327p1-p950708.html',
 'https://xwatch.vn/dong-ho-nu/opa28022dlsk-t-p2508.html',
 'https://xwatch.vn/dong-ho-nam/ap1052-00a-p5572.html',
 'https://xwatch.vn/dong-ho-nam/c43634-p957949.html',
 'https://xwatch.vn/dong-ho-nu/mk6588-p953862.html',
 'https://xwatch.vn/dong-ho-nu/opa28019dls-gl-t-p2919.html',
 'https://xwatch.vn/dong-ho-nu/opa58012-07dlk-t-p2142.html',
 'https://xwatch.vn/dong-ho-nu/fc-200whd1er6b-p958690.html',
 'https://xwatch.vn/dong-ho-nam/orient-se-ra-ag0430s00b-p957165.html',
 'https://xwatch.vn/dong-ho-nam/op9921-77amsr-t-p957392.html',
 'https://xwatch.vn/dong-ho-nam/ak5000-03a-p958086.html',
 'https://xwatch.vn/dong-ho-nam/ra-ac0e03s10b-p957445.html',
 'https://xwatch.vn/dong-ho-nu/srz384p1-p950870.html',
 'https://xwatch.vn/dong-ho-nam/ssb303p1-p957419.html',
 'https://xwatch.vn/dong-ho-nu/el3040-12d-p9536

In [4]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup


soup = get_soup(all_links[0])
details = {}
nameAndtype = soup.find("div", class_="product_name")
ds = nameAndtype.getText().lower().replace("chính hãng","").strip().split(" ")
type = ds[-1].strip().upper()
price = (
    soup.find_all("h3", class_="price_current")[0]
    .text.replace("₫", "")
    .strip()
)

content = soup.find("div", class_="table-condensed compare_table")
rows = content.find_all("tr")
for row in rows:
    text = row.getText().replace("\t", "").replace("\n", "").replace("\r", "")
    if len(text.split(":")) == 2:
        key, value = text.split(":")
        details[key.lower().strip()] = value.strip()
a = {"Giá tiền": price, "Mã sản phẩm": type, **details}
print(a)

{'Giá tiền': '6.780.000', 'Mã sản phẩm': 'BM7141-51A', 'nhãn hiệu': 'Citizen', 'nguồn gốc': 'Nhật Bản', 'kiểu máy': 'Eco-Drive', 'kích cỡ': '41mm', 'chất liệu vỏ': 'Thép không gỉ', 'chất liệu dây': 'Thép không gỉ', 'chất liệu kính': 'Kính Sapphire', 'độ chịu nước': '50M', 'bảo hiểm': '5 năm cả lỗi người dùng tại Xwatch', 'bảo hành quốc tế': '1 năm tiêu chuẩn đồng hồ chính hãng', 'giới tính': 'Nam'}


In [5]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup
def get_product_info(url):
    soup = get_soup(url)
    details = {}
    nameAndtype = soup.find("div", class_="product_name")
    ds = nameAndtype.getText().lower().replace("chính hãng","").strip().split(" ")
    type = ds[-1].strip().upper()
    price = (
        soup.find_all("h3", class_="price_current")[0]
        .text.replace("₫", "")
        .strip()
    )

    content = soup.find("div", class_="table-condensed compare_table")
    rows = content.find_all("tr")
    for row in rows:
        text = row.getText().replace("\t", "").replace("\n", "").replace("\r", "")
        if len(text.split(":")) == 2:
            key, value = text.split(":")
            details[key.strip()] = value.strip()
    a = {"Giá tiền": price, "Mã sản phẩm": type, **details}
    print(a)
    return {"Giá tiền": price, "Mã sản phẩm": type, **details}

In [6]:
def crawl_data_from_web(all_links):
    data_frame_watchs = pd.DataFrame()
    with ThreadPoolExecutor(max_workers=25) as executor:
        results = executor.map(get_product_info, all_links)
        for result in results:
            data_frame_watchs = data_frame_watchs.append(result, ignore_index=True)
    return data_frame_watchs

data_frame_watchs = crawl_data_from_web(all_links)

data_frame_watchs

# data_frame_watchs.to_csv("../main/raw_data/x_watch.csv")

{'Giá tiền': '6.780.000', 'Mã sản phẩm': 'BM7141-51A', 'Nhãn hiệu': 'Citizen', 'Nguồn gốc': 'Nhật Bản', 'Kiểu máy': 'Eco-Drive', 'Kích cỡ': '41mm', 'Chất liệu vỏ': 'Thép không gỉ', 'Chất liệu dây': 'Thép không gỉ', 'Chất liệu kính': 'Kính Sapphire', 'Độ chịu nước': '50M', 'Bảo hiểm': '5 năm cả lỗi người dùng tại Xwatch', 'Bảo hành quốc tế': '1 năm tiêu chuẩn đồng hồ chính hãng', 'Giới tính': 'Nam'}
{'Giá tiền': '3.530.000', 'Mã sản phẩm': 'AK5000-03A', 'Nhãn hiệu': 'Citizen', 'Nguồn gốc': 'Nhật Bản', 'Kiểu máy': 'Quartz (Máy pin)', 'Kích cỡ': '41mm', 'Độ dày': '10mm', 'Chất liệu vỏ': 'Thép không gỉ 316L', 'Chất liệu dây': 'Dây da', 'Chất liệu kính': 'Kính cứng', 'Độ chịu nước': '50M', 'Chức năng khác': 'Lịch Trăng Sao', 'Bảo hiểm': '5 năm cả lỗi người dùng tại Xwatch', 'Bảo hành quốc tế': '1 năm', 'Giới tính': 'Nam'}
{'Giá tiền': '5.820.000', 'Mã sản phẩm': 'SRZ384P1', 'Nhãn hiệu': 'Seiko', 'Nguồn gốc': 'Nhật Bản', 'Kiểu máy': 'Quartz', 'Kích cỡ': '28 mm', 'Chất liệu vỏ': 'Thép - PVD v

,Giá tiền,Mã sản phẩm,Nhãn hiệu,Nguồn gốc,Kiểu máy,Kích cỡ,Chất liệu vỏ,Chất liệu dây,Chất liệu kính,Độ chịu nước,Bảo hiểm,Bảo hành quốc tế,Giới tính,Chức năng khác,Độ dày,Thương hiệu
0,6.780.000,BM7141-51A,Citizen,Nhật Bản,Eco-Drive,41mm,Thép không gỉ,Thép không gỉ,Kính Sapphire,50M,5 năm cả lỗi người dùng tại Xwatch,1 năm tiêu chuẩn đồng hồ chính hãng,Nam,NaN,NaN,NaN
1,8.380.000,SRPD71K1,Seiko,Nhật Bản,Automatic,42.5 mm,Thép không gỉ 316L,Dây mesh,Harlex Seiko,100 m,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,"Lịch ngày, lịch thứ",NaN,NaN
2,5.650.000,SSB327P1,Seiko,Nhật Bản,Quartz,44mm,Thép - PVD Ion đen,Dây cao su,Kính cứng,100M,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,Bấm giờ,NaN,NaN
3,4.330.000,OPA28022DLSK-T,Olympia Star,Thụy Sỹ,Swiss quartz (chạy pin),NaN,Thép không gỉ 316L,Thép không gỉ 316L,Sapphire,30m,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nữ,NaN,NaN,NaN
4,12.500.000,AP1052-00A,Citizen,Nhật Bản,Eco-Drive,40mm,Thép không gỉ,Dây da,Sapphire,50m,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3727,7.470.000,FDD03001W0,Orient,Nhật Bản,Lên cót tay,40 mm,Thép không gỉ 316L,Dây da,Kính Cứng,50 m,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,NaN,NaN,NaN
3728,7.230.000,SSB363P1,Seiko,Nhật Bản,Quartz,45 mm,Thép không gỉ 316L,Thép không gỉ 316L,Kính cứng,100 m,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,"Lịch ngày, Chronograph",NaN,NaN
3729,5.600.000,AU1080-11L,Citizen,Nhật Bản,Eco-Drive,38 mm,Thép không gỉ 316L/Mạ PVD,Dây da,Kính Sapphire,30 m,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,NaN,NaN,NaN
3730,8.980.000,RA-AP0004S10B,Orient,Nhật Bản,Automatic,"40,5 mm",Thép - PVD vàng,Dây da,Kính cứng,30M,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,NaN,NaN,NaN
